In [ ]:
import numpy as np
from sklearn import preprocessing
import random
import math
import matplotlib.pyplot as plt 
from sklearn.model_selection import train_test_split
import cv2
from google.colab import drive
from scipy.stats import multivariate_normal as gaussian
import os
import glob

In [ ]:
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
video = cv2.VideoCapture('/content/drive/MyDrive/PRP3/Car_less_resolution.mp4')

In [ ]:
from IPython.display import HTML
from base64 import b64encode
 
def show_video(video_path, video_width = 600):
   
  video_file = open(video_path, "r+b").read()
 
  video_url = f"data:video/mp4;base64,{b64encode(video_file).decode()}"
  return HTML(f"""<video width={video_width} controls><source src="{video_url}"></video>""")

In [ ]:
show_video('/content/drive/MyDrive/PRP3/Car_less_resolution.mp4')

Convering video to frames

In [ ]:
def FrameCapture(path):
    vidObj = cv2.VideoCapture(path)
    count = 0
    success = 1
    frames = np.zeros((240,426,138))
    while success:
        # vidObj object calls read
        # function extract frames
        success, image = vidObj.read()
        if(success):
            # Saves the frames with frame-count
            img_gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            frames[:,:,count] = img_gray
            cv2.imwrite("/content/drive/MyDrive/PRP3/Frames/b"+str(count)+".jpg",img_gray)
            count += 1
    return frames


In [ ]:
cwd = "/content/drive/MyDrive/PRP3/Car_less_resolution.mp4"
frames = FrameCapture(cwd)

In [ ]:
path_1 = '/content/drive/MyDrive/PRP3/Frames/b58.jpg'
image1 = cv2.imread(path_1)
image1.shape

(240, 426, 3)

Here we go

In [ ]:
def gaussiandef(x, mean, s):
  mean = np.atleast_2d(mean)
  s = np.atleast_2d(s)
  N = x.size

  temp1 = np.linalg.det(s) ** (-0.5)
  temp2 = np.exp(-0.5 * ((x - mean).T) @ np.linalg.inv(s) @ (x - mean))
  return (2 * np.pi) ** (-N/2) * temp1 * temp2

In [ ]:
# # # Makes a list of paths of all frames
images = glob.glob("/content/drive/MyDrive/PRP3/Frames/b*.jpg")
images = sorted(images)
print(f'Total number of images: {len(images)}')


Total number of images: 138


In [ ]:
train_set_len = 138
train_set = images[:train_set_len]

In [ ]:
example_img = cv2.imread(train_set[0])
height, width, channels = example_img.shape
print(f'Dimensions of an Image: {height} x {width} x {channels}') # # # Dimensions of each image

Dimensions of an Image: 240 x 426 x 3


In [ ]:
alpha = 1.0/train_set_len # # Defining the parameter 'alpha'
K = 2 # # Parameter K defined in the paper (number of Gaussians)
thres1 = 2.5 # # Threshold to classify a pixel as background or foreground
forethres = 0.5 # # Threshold for the Euclidean Distance (to check if it has significant weight in representation of the pixel)

In [ ]:
B = np.ones((height, width),dtype=np.int64) # # Matrix for background pixels
weights = np.ndarray(shape=(height,width, K), dtype=np.int64) # # Weight matrix to represent the weights of each of the Gaussian
for h in range(height):
  for w in range(width):
    weights[h,w,:] = [0.5,0.5]  # # # Initial weights

print(f'Shape of weight array: {weights.shape}')

Shape of weight array: (240, 426, 2)


In [ ]:
# # # Defining the expectation (or the mean) matrix
exp = np.ndarray(shape=(height,width,K,channels), dtype = np.float64)
for h in range(height):
  for w in range(width):
    for k in range(K):
      exp[h,w,k,:] = np.zeros(3)
      exp[h][w][k] = np.array(example_img[h][w]).reshape(1,3)

print(f'Shape of expectation(mean) array: {exp.shape}')

Shape of expectation(mean) array: (240, 426, 2, 3)


In [ ]:
# # # Defining the variance matrix
sigma = np.ndarray(shape=(height,width,K,channels, channels), dtype = np.float64)
for h in range(height):
  for w in range(width):
    for k in range(K):
      sigma[h,w,k,:,:] = 255*np.eye(channels)

print(f'Shape of standard deviation array: {sigma.shape}')

Shape of standard deviation array: (240, 426, 2, 3, 3)


In [ ]:
# # # Using the first threshold, checks if the pixel could be classified as background or foreground.
def match_check(x, exp, sigma):
    m = np.mat(np.reshape(x, (3, 1)))
    n = np.mat(exp).T
    sigma = np.mat(sigma)
    d = np.sqrt((m-n).T*sigma.I*(m-n))
    if d < thres1:
        return True
    else:
        return False

In [ ]:
for training_image in train_set:
  image = cv2.imread(training_image)
  for h in range(height):
    for w in range(w):
      flag = -1
      for k in range(K):
        if (match_check(image[h][w], exp[h][w][k], sigma[h][w][k])) is True:
          flag = k
          break
      m = np.array(image[h][w]).reshape(1,3)

      if flag != 1:
        updated_exp = exp[h][w][flag]
        updated_sigma = sigma[h][w][flag]
        m = image[h][w].astype(np.float)
        p = gaussian.pdf(image[h][w], updated_exp, updated_sigma)
        # p = gaussiandef(image[h][w], updated_exp, updated_sigma)
        weights[h][w] = (1-alpha)*weights[h][w]
        weights[h][w][flag] += alpha
        exp[h][w][flag] += (m - updated_exp)*p
        sigma[h][w][flag] += p*(np.matmul(m-updated_exp, (m-updated_exp).T) - updated_sigma)

      else:
        w = [weights[h][w][k] for k in range(K)]
        minindex = w.index(min(w))
        exp[h][w][minindex] = m
        sigma[h][w][minindex] = 255*(np.eye(3))

  for h in range(height):
        for w in range(width):
            rank = weights[h][w]*1.0/[np.linalg.norm(np.sqrt(sigma[h][w][k])) for k in range(K)]
            rank_ind = [k for k in range(K)]
            rank_ind.sort(key=lambda x: -rank[x])
            weights[h][w] = weights[h][w][rank_ind]
            exp[h][w] =  exp[h][w][rank_ind]
            sigma[h][w] = sigma[h][w][rank_ind]
            cum = 0
            for ind, order in enumerate(rank_ind):
                cum += weights[h][w][ind]
                if cum > forethres:
                    B[h][w] = ind + 1
                    break
                else:
                    break

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:15: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  from ipykernel import kernelapp as app


In [ ]:
def infer(img):
      result = np.array(img)
      for i in range(height):
          for j in range(width):
              for k in range(B[i][j]):
                  if match_check(img[i][j], exp[i][j][k], sigma[i][j][k]):
                      result[i][j] = [0, 0, 0]    
                      break
                  else:
                      result[i][j] = [255, 255, 255]    
                      break
      return result

In [ ]:
file_list = glob.glob(r'/content/drive/MyDrive/PRP3/Frames/b*.jpg')
for index, file in enumerate(file_list):
        # print('infering:{}'.format(file))
        img = cv2.imread(file)
        img2 = infer(img)
        cv2.imwrite('/content/drive/MyDrive/PRP3/output/b'+'%05d'%index+'.jpg', img2)
        index += 1

Converting frames to videos. The video will appear in the files of colab. Download and enjoy:)

In [99]:
img_array = []
for filename in glob.glob('/content/drive/MyDrive/PRP3/output/b*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('output.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

Comparision with inbuilt functions

In [ ]:
file_list = glob.glob(r'/content/drive/MyDrive/PRP3/Frames/b*.jpg')
fgbg0 = cv2.bgsegm.createBackgroundSubtractorMOG()
for index, file in enumerate(file_list):
        # print('infering:{}'.format(file))
        img = cv2.imread(file)
        img1 = fgbg0.apply(img)
        cv2.imwrite('/content/drive/MyDrive/PRP3/Output_inbuilt_MOG/b'+'%05d'%index+'.jpg', img1)
        index += 1

In [ ]:
file_list = glob.glob(r'/content/drive/MyDrive/PRP3/Frames/b*.jpg')
fgbg = cv2.createBackgroundSubtractorMOG2()
for index, file in enumerate(file_list):
        # print('infering:{}'.format(file))
        img = cv2.imread(file)
        img2 = fgbg.apply(img)
        cv2.imwrite('/content/drive/MyDrive/PRP3/Output_inbuilt_MOG2/b'+'%05d'%index+'.jpg', img2)
        index += 1

In [100]:
img_array = []
for filename in glob.glob('/content/drive/MyDrive/PRP3/Output_inbuilt_MOG/b*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('output_MOG.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [101]:
img_array = []
for filename in glob.glob('/content/drive/MyDrive/PRP3/Output_inbuilt_MOG2/b*.jpg'):
    img = cv2.imread(filename)
    height, width, layers = img.shape
    size = (width,height)
    img_array.append(img)
 
 
out = cv2.VideoWriter('output_MOG2.mp4',cv2.VideoWriter_fourcc(*'DIVX'), 15, size)
 
for i in range(len(img_array)):
    out.write(img_array[i])
out.release()

In [87]:
def mse(imageA, imageB):
	# the 'Mean Squared Error' between the two images is the
	# sum of the squared difference between the two images;
	# NOTE: the two images must have the same dimension
	err = np.sum((imageA.astype("float") - imageB.astype("float")) ** 2)
	err /= float(imageA.shape[0] * imageA.shape[1]*imageA.shape[2])*3
 
	
	# return the MSE, the lower the error, the more "similar"
	# the two images are
	return err